<a href="https://colab.research.google.com/github/Vu529/P.h.D-AI-project/blob/main/TB_drug_cytotoxicity_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving TB.xlsx to TB.xlsx


In [3]:
import io
df2 = pd.read_excel(io.BytesIO(uploaded['TB.xlsx']))

In [2]:
import pandas as pd

df2 = pd.read_excel('TB.xlsx')
df2

,Class,canonical_smiles
0,Active,CC1=CC(=CC(=C1)CN2C(=O)C(=C(C=N2)Cl)Cl)C
1,Active,CC1=CC=CC=C1N2C3=C(C=N2)C(=CC(=C3)[N+](=O)[O-]...
2,Active,C1=CSC(=C1)C=C2C(=O)N(C(=S)S2)CCCC(=O)NC3=CC=C...
3,Active,CC1=CC=C(C=C1)CS(=O)(=O)C2=NC=C(C(=N2)C(=O)NC3...
4,Active,CC(C)C1=NN=C(S1)NC(=O)C2=NC(=NC=C2Cl)S(=O)(=O)...
...,...,...
125,Inactive,CN1C(=C(C(=O)N(C1=O)C)C2=NN(C(C2)C3=CC=CC=C3)C...
126,Inactive,CC1=C(SC=C1)C=NNC(=O)CC2=CC(=O)NN2
127,Inactive,C1=CC(=CC=C1C2=CC(=O)NN2)Br
128,Inactive,CC(=NNC(=O)CC1=CC(=O)NN1)C2=CC(=CC=C2)[N+](=O)...


In [4]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2022-07-26 13:56:45--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   104MB/s    in 0.8s    

2022-07-26 13:56:46 (104 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h

# Calculate Lipinski descriptors Christopher Lipinski, a scientist at Pfizer, came up with a set of rule-of-thumb for evaluating the druglikeness of compounds. Such druglikeness is based on the Absorption, Distribution, Metabolism and Excretion (ADME) that is also known as the pharmacokinetic profile. Lipinski analyzed all orally active FDA-approved drugs in the formulation of what is to be known as the Rule-of-Five or Lipinski's Rule.

In [7]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [5]:
def lipinski(smiles, verbose=False):

    moldata= []
    for elem in smiles:
        mol=Chem.MolFromSmiles(elem) 
        moldata.append(mol)
       
    baseData= np.arange(1,1)
    i=0  
    for mol in moldata:        
       
        desc_MolWt = Descriptors.MolWt(mol)
        desc_MolLogP = Descriptors.MolLogP(mol)
        desc_NumHDonors = Lipinski.NumHDonors(mol)
        desc_NumHAcceptors = Lipinski.NumHAcceptors(mol)
           
        row = np.array([desc_MolWt,
                        desc_MolLogP,
                        desc_NumHDonors,
                        desc_NumHAcceptors])   
    
        if(i==0):
            baseData=row
        else:
            baseData=np.vstack([baseData, row])
        i=i+1      
    
    columnNames=["MW","LogP","NumHDonors","NumHAcceptors"]   
    descriptors = pd.DataFrame(data=baseData,columns=columnNames)
    
    return descriptors

In [8]:
df_lipinski = lipinski(df2.canonical_smiles)
df_lipinski

,MW,LogP,NumHDonors,NumHAcceptors
0,283.158,3.21524,0.0,3.0
1,298.258,3.15032,0.0,6.0
2,404.538,4.07380,2.0,6.0
3,451.961,3.63952,1.0,8.0
4,455.924,3.47020,1.0,8.0
...,...,...,...,...
125,376.800,0.70610,1.0,7.0
126,264.310,0.76572,3.0,4.0
127,239.072,2.13250,2.0,1.0
128,303.278,0.69410,3.0,5.0


In [9]:
df1_combined =  df2.drop(columns='canonical_smiles')

In [12]:
df2_combined = pd.concat([df1_combined, df_lipinski], axis=1)
df2_combined

,Class,MW,LogP,NumHDonors,NumHAcceptors
0,Active,283.158,3.21524,0.0,3.0
1,Active,298.258,3.15032,0.0,6.0
2,Active,404.538,4.07380,2.0,6.0
3,Active,451.961,3.63952,1.0,8.0
4,Active,455.924,3.47020,1.0,8.0
...,...,...,...,...,...
125,Inactive,376.800,0.70610,1.0,7.0
126,Inactive,264.310,0.76572,3.0,4.0
127,Inactive,239.072,2.13250,2.0,1.0
128,Inactive,303.278,0.69410,3.0,5.0


PadelPy installation and calculation of molecular fingerprints

In [ ]:
! pip install padelpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 20.9 MB 1.2 MB/s 


In [ ]:
! wget https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip
! unzip fingerprints_xml.zip

--2022-07-24 20:53:36--  https://github.com/dataprofessor/padel/raw/main/fingerprints_xml.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dataprofessor/padel/main/fingerprints_xml.zip [following]
--2022-07-24 20:53:36--  https://raw.githubusercontent.com/dataprofessor/padel/main/fingerprints_xml.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10871 (11K) [application/zip]
Saving to: ‘fingerprints_xml.zip’

fingerprints_xml.zi 100%[===================>]  10.62K  --.-KB/s    in 0s      

2022-07-24 20:53:36 (96.4 MB/s) - ‘fingerprints_xml.zip’ saved [10871/10871]

Archive:  fingerprints_xm

In [ ]:
import glob
xml_files = glob.glob("*.xml")
xml_files.sort()
xml_files

['AtomPairs2DFingerprintCount.xml',
 'AtomPairs2DFingerprinter.xml',
 'EStateFingerprinter.xml',
 'ExtendedFingerprinter.xml',
 'Fingerprinter.xml',
 'GraphOnlyFingerprinter.xml',
 'KlekotaRothFingerprintCount.xml',
 'KlekotaRothFingerprinter.xml',
 'MACCSFingerprinter.xml',
 'PubchemFingerprinter.xml',
 'SubstructureFingerprintCount.xml',
 'SubstructureFingerprinter.xml']

In [ ]:
FP_list = ['AtomPairs2DCount',
           'AtomPairs2D',
           'EState',
           'CDKextended',
           'CDK',
           'CDKgraphonly',
           'KlekotaRothCount',
           'KlekotaRoth',
           'MACC',
           'Pubchem',
           'SubstructureCount',
           'Substructure']

In [ ]:
fp = dict(zip(FP_list, xml_files))
fp

{'AtomPairs2D': 'AtomPairs2DFingerprinter.xml',
 'AtomPairs2DCount': 'AtomPairs2DFingerprintCount.xml',
 'CDK': 'Fingerprinter.xml',
 'CDKextended': 'ExtendedFingerprinter.xml',
 'CDKgraphonly': 'GraphOnlyFingerprinter.xml',
 'EState': 'EStateFingerprinter.xml',
 'KlekotaRoth': 'KlekotaRothFingerprinter.xml',
 'KlekotaRothCount': 'KlekotaRothFingerprintCount.xml',
 'MACC': 'MACCSFingerprinter.xml',
 'Pubchem': 'PubchemFingerprinter.xml',
 'Substructure': 'SubstructureFingerprinter.xml',
 'SubstructureCount': 'SubstructureFingerprintCount.xml'}

In [ ]:
selection = ['canonical_smiles','Class']
df3_selection = df2[selection]
df3_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [ ]:
! cat molecule.smi | head -5

CC1=CC(=CC(=C1)CN2C(=O)C(=C(C=N2)Cl)Cl)C	Active
CC1=CC=CC=C1N2C3=C(C=N2)C(=CC(=C3)[N+](=O)[O-])[N+](=O)[O-]	Active
C1=CSC(=C1)C=C2C(=O)N(C(=S)S2)CCCC(=O)NC3=CC=C(C=C3)O	Active
CC1=CC=C(C=C1)CS(=O)(=O)C2=NC=C(C(=N2)C(=O)NC3=NN=C(S3)C(C)C)Cl	Active
CC(C)C1=NN=C(S1)NC(=O)C2=NC(=NC=C2Cl)S(=O)(=O)CC3=CC=C(C=C3)F	Active


In [ ]:
! cat molecule.smi | wc -l

130


In [ ]:
from padelpy import padeldescriptor
fingerprint = 'Substructure'
fingerprint_output_file = ''.join([fingerprint, '.excel']) #'Substructure.csv'
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='molecule.smi',
                  d_file=fingerprint_output_file,#'Substructure.csv'
                  #descriptortypes='SubstructureFingerprint.xml'),
                  descriptortypes= fingerprint_descriptortypes,
                  detectaromaticity=True,
                  standardizenitro=True,
                  standardizetautomers=True,
                  threads=2,
                  removesalt=True,
                  log=True,
                  fingerprints=True)

In [ ]:
from padelpy import padeldescriptor
fingerprint = 'MACC'
MACC_output_file = ''.join([fingerprint, '.excel']) #'MACC.csv'
fingerprint_descriptortypes = fp[fingerprint]

padeldescriptor(mol_dir='molecule.smi',
                  d_file=fingerprint_output_file,#'MACC.csv'
                  #descriptortypes='MACCFingerprint.xml'),
                  descriptortypes= fingerprint_descriptortypes,
                  detectaromaticity=True,
                  standardizenitro=True,
                  standardizetautomers=True,
                  threads=2,
                  removesalt=True,
                  log=True,
                  fingerprints=True)

In [ ]:
Descriptors1 =pd.read_csv(MACC_output_file)
Descriptors1

,Name,MACCSFP1,MACCSFP2,MACCSFP3,MACCSFP4,MACCSFP5,MACCSFP6,MACCSFP7,MACCSFP8,MACCSFP9,...,MACCSFP157,MACCSFP158,MACCSFP159,MACCSFP160,MACCSFP161,MACCSFP162,MACCSFP163,MACCSFP164,MACCSFP165,MACCSFP166
0,Active,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0
1,Active,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
2,Active,0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,1,0
3,Active,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
4,Active,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,Inactive,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,1,1,0
126,Inactive,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,1,1,1,1,0
127,Inactive,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,1,1,1,1,0
128,Inactive,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0


In [ ]:
df_MACC = Descriptors1.drop(columns='Name')

In [ ]:
Descriptors= pd.read_csv(fingerprint_output_file)
Descriptors

,Name,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,SubFP6,SubFP7,SubFP8,SubFP9,...,SubFP298,SubFP299,SubFP300,SubFP301,SubFP302,SubFP303,SubFP304,SubFP305,SubFP306,SubFP307
0,Active,1,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,0,0,0,1
1,Active,1,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,0,0,1
2,Active,0,1,0,0,0,0,0,0,0,...,0,0,1,1,1,1,0,0,0,1
3,Active,1,0,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
4,Active,1,0,1,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,Inactive,0,1,1,0,0,0,0,1,0,...,0,0,1,1,1,0,0,0,0,1
126,Inactive,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
127,Inactive,1,1,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,1
128,Inactive,1,1,0,0,0,0,0,0,0,...,1,1,1,1,1,0,0,0,0,1


In [ ]:
df_combined = pd.concat([df_lipinski, df2], axis=1)
df_combined

,MW,LogP,NumHDonors,NumHAcceptors,Name,SubFP1,SubFP2,SubFP3,SubFP4,SubFP5,...,MACCSFP157,MACCSFP158,MACCSFP159,MACCSFP160,MACCSFP161,MACCSFP162,MACCSFP163,MACCSFP164,MACCSFP165,MACCSFP166
0,283.158,3.21524,0.0,3.0,Active,1,0,0,0,0,...,0,1,0,1,1,1,1,1,1,0
1,298.258,3.15032,0.0,6.0,Active,1,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
2,404.538,4.07380,2.0,6.0,Active,0,1,0,0,0,...,1,1,1,0,1,1,1,1,1,0
3,451.961,3.63952,1.0,8.0,Active,1,0,1,0,0,...,0,1,1,1,1,1,1,1,1,0
4,455.924,3.47020,1.0,8.0,Active,1,0,1,0,0,...,0,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,376.800,0.70610,1.0,7.0,Inactive,0,1,1,0,0,...,1,1,1,1,1,1,0,1,1,0
126,264.310,0.76572,3.0,4.0,Inactive,0,0,0,0,0,...,1,0,1,0,1,1,1,1,1,0
127,239.072,2.13250,2.0,1.0,Inactive,1,1,0,0,0,...,1,0,0,0,1,1,1,1,1,0
128,303.278,0.69410,3.0,5.0,Inactive,1,1,0,0,0,...,1,1,1,1,1,1,1,1,1,0


In [ ]:
df_combined.to_csv('TB_cytotoxicity.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
! mkdir '/content/gdrive/My Drive/Colab Notebooks/dataTB'

In [ ]:
! cp TB_cytotoxicity.csv '/content/gdrive/My Drive/Colab Notebooks/dataTB'

In [ ]:
! ls -l '/content/gdrive/My Drive/Colab Notebooks/dataTB'

total 131
-rw------- 1 root root 133857 Jul 24 22:07 TB_cytotoxicity.csv
